<a href="https://colab.research.google.com/github/ritterl/MachineLearning/blob/master/Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext rpy2.ipython

# Introduction

In this notebook, chapter 10 of [Machine Learning](https://www.amazon.de/Machine-Learning-techniques-predictive-modeling-ebook/dp/B07PYXX3H5) with R is summarized and the code samples are described. This section of the book serves to show how machine learning algorithms can be evaluated. More precisely, it gives reasons why also other measures than predictive accuracy are needed to assess performance. Further, approaches to ensure that the performance measures reasonably reflect a model's ability to predict or forecast unseen cases are provided. 

By simply dividing the number of correct predictions by the total number of predictions, a wrong picture about the performance of the classifier may occur. This especially arises in datasets with a large class imbalance and is also referred to as **class imbalance problem**. For instance, a positive event occurs very often (in say 99% of the cases), a model which predicts always a positive case has an accuracy of 99%. However, it is not useful for predicting the negative cases. 

Besides the two important data types: **actual class values** and **predicted class values** which are obvious, however, the majority of models can deliver another important type of information: the **estimated probability of the prediction** or in other words the confidence of the model about a particular decision. So, when comparing two models with the same number of mistakes, it is possible to say that the one which makes better assessments regarding its uncertainty is smarter.

In [1]:
# obtain the predicted probabilities
sms_test_prob <- predict(sms_classifier, sms_test, type = "raw")
head(sms_test_prob)

ERROR! Session/line number was not unique in database. History logging moved to new session 59


NameError: ignored